In [ ]:
%load_ext autoreload
%autoreload 2

## Libraries

# Variable factory

In [21]:
# Add root to path
from typing import Literal
import sys
import ee
import json
sys.path.append("..")
from component.script.gee.ee_fao_gaul import get_fao_gaul_features, get_fao_gaul_subj
from component.script.variables.models import RasterizationMethod
from component.script.variables import LocalVectorVar, LocalRasterVar, GEEVar
from component.script.project import Project
from component.script.gee.ee_rasterize_unique_values import gee_rasterize_unique_values

## GEE


In [22]:
ee_project = "ee-dfgm2006"
ee.Initialize(project=ee_project)

## Set project parameters

## AOI

In [23]:

# The model_rebuild() calls happen automatically in both modules,
# but we ensure the imports are done in the right order

project_name = "nuevo2"
years = [2015, 2020, 2024]

project = Project(
    project_name=project_name,
    years=years
)

In [24]:
# Check the project variables (have to be empty)
project.variables

{}

In [25]:
# We can save the project
project.save()

Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


In [26]:
aoi_source: Literal["gee", "local"] = "local"

In [27]:
if aoi_source=="gee":

    iso_code = "MTQ"
    aoi_image = get_fao_gaul_features(level=0, code=iso_code)

    aoi_var = GEEVar(
        name="aoi",
        data_type="vector",
        gee_images=[aoi_image],
        project=project,
        aoi=aoi_image.geometry(),
    )

    # Download and convert to local vector
    aoi_local = aoi_var.to_local_vector()
    aoi = aoi_image


else:
    aoi_local_var = LocalVectorVar(
        path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/Limite_SabanasYari/Limite_SabanasYari/NAD_sabanasYari.shp",
        name="aoi",
        project=project,
    )

    aoi = aoi_local_var.to_gee_var()

## SubJuridistion

In [28]:
filtered_subj, filtered_attribute = get_fao_gaul_subj(2, aoi)
subj_image = (
    ee.Image(gee_rasterize_unique_values(filtered_subj, "gaul2_name"))
    .clip(aoi)
    .toByte()
)

subj_var = GEEVar(
    name="subj",
    data_type="raster",
    raster_type="categorical",
    gee_images=[subj_image],
    aoi=aoi,
    project=project,
    # tags=["forest_loss"]

)
subj_var_local = subj_var.to_local_raster() # Returns a LocalVar instance
subj_var_local.add_as_raw();

  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/subj.tif, downloaded
✓ Added 'subj' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/.venv/lib/python3.11/site-packages/geedim/image.py:254: RuntimeWarning: Couldn't find STAC entry for: 'None'.
  return STACClient().get(self.id)


# GEE VARIABLES

## Areas protegidas

In [29]:
wdpa_poly = (
    ee.FeatureCollection("WCMC/WDPA/current/polygons")
    .filterBounds(aoi)
    .filter(
        ee.Filter.inList(
            "STATUS", ["Designated", "Inscribed", "Established", "Proposed"]
        )
    )
)
wdpa_image = (
    wdpa_poly.reduceToImage(["WDPAID"], ee.Reducer.first())
    .gt(0)
    .unmask()
    .clip(aoi)
    .toByte()
)

protected_area_var = GEEVar(
    name="protected_area",
    data_type="raster",
    raster_type="categorical",
    gee_images=[wdpa_image],
    aoi=aoi,
    project=project
)

protected_area_var_local = protected_area_var.to_local_raster() # Returns a LocalVar instance
protected_area_var_local.add_as_raw();

  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/protected_area.tif, downloaded
✓ Added 'protected_area' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


## Altitude

In [30]:
altitude_image = ee.Image("USGS/SRTMGL1_003").select("elevation").clip(aoi)
altitude_var = GEEVar(
    name="altitude",
    data_type="raster",
    raster_type="continuous",
    gee_images=[altitude_image],
    aoi=aoi,
    project=project
)
altitude_var_local = altitude_var.to_local_raster() # Returns a LocalVar instance
altitude_var_local.add_as_raw();

USGS/SRTMGL1_003:   0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/altitude.tif, downloaded
✓ Added 'altitude' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


## Slope

In [31]:
slope_image = ee.Terrain.slope(altitude_image).clip(aoi)
slope_var = GEEVar(
    name="slope",
    data_type="raster",
    raster_type="continuous",
    gee_images=[slope_image],
    aoi=aoi,
    project=project
)
slope_var_local = slope_var.to_local_raster()
slope_var_local.add_as_raw();

  0%|          |0/64 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/slope.tif, downloaded
✓ Added 'slope' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


## Forest layers

In [32]:

forest_source = "gfc"  # "gfc" or "tmf"
tree_cover_threshold = 10  # in percentage


if forest_source == "gfc":
    gfcImage = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")
    clipGfc = gfcImage.clip(aoi)
    forest2000 = clipGfc.select(["treecover2000"])
    forest2000_thr = (
        ee.Image(0).where(forest2000.gte(tree_cover_threshold), 1).clip(aoi)
    )
    loss = clipGfc.select(["lossyear"])

    forest_gcf_t1 = forest2000_thr.where(loss.lt(years[0] - 2000), 0).rename("B1")
    forest_gcf_t2 = forest2000_thr.where(loss.lt(years[1] - 2000), 0).rename("B1")
    forest_gcf_t3 = forest2000_thr.where(loss.lt(years[2] - 2000), 0).rename("B1")

    # Create separate GEEVar for each time period - explicit and clear
    forest_change_var_t1 = GEEVar(
        name=f"forest_gfc_{tree_cover_threshold}_{years[0]}",
        data_type="raster",
        raster_type="categorical",
        gee_images=[forest_gcf_t1],
        aoi=aoi,
        project=project,
        year=years[0],
        tags=["forest"]
    )
    
    forest_change_var_t2 = GEEVar(
        name=f"forest_gfc_{tree_cover_threshold}_{years[1]}",
        data_type="raster",
        raster_type="categorical",
        gee_images=[forest_gcf_t2],
        aoi=aoi,
        project=project,
        year=years[1],
        tags=["forest"]
    )
    
    forest_change_var_t3 = GEEVar(
        name=f"forest_gfc_{tree_cover_threshold}_{years[2]}",
        data_type="raster",
        raster_type="categorical",
        gee_images=[forest_gcf_t3],
        aoi=aoi,
        project=project,
        year=years[2],
        tags=["forest"]
    )
    
    forest_change_vars = [forest_change_var_t1, forest_change_var_t2, forest_change_var_t3]

elif forest_source == "tmf":
    tmfImage = (
        ee.ImageCollection("projects/JRC/TMF/v1_2024/AnnualChanges")
        .filterBounds(aoi)
        .mosaic()
    )
    forest2_t1 = tmfImage.select("Dec" + str(years[0] - 1))
    forest2_t2 = tmfImage.select("Dec" + str(years[1] - 1))
    forest2_t3 = tmfImage.select("Dec" + str(years[2] - 1))

    forest_tmf_t1 = (
        forest2_t1.where(forest2_t1.eq(2), 1).where(forest2_t1.neq(1), 0).rename("B1")
    )
    forest_tmf_t2 = (
        forest2_t2.where(forest2_t2.eq(2), 1).where(forest2_t2.neq(1), 0).rename("B2")
    )
    forest_tmf_t3 = (
        forest2_t3.where(forest2_t3.eq(2), 1).where(forest2_t3.neq(1), 0).rename("B3")
    )

    # Create separate GEEVar for each time period - explicit and clear
    forest_change_var_t1 = GEEVar(
        name=f"forest_tmf_{years[0]}",
        data_type="raster",
        raster_type="categorical",
        gee_images=[forest_tmf_t1],
        aoi=aoi,
        project=project,
        year=years[0],
        tags=["forest"]
    )
    
    forest_change_var_t2 = GEEVar(
        name=f"forest_tmf_{years[1]}",
        data_type="raster",
        raster_type="categorical",
        gee_images=[forest_tmf_t2],
        aoi=aoi,
        project=project,
        year=years[1],
        tags=["forest"]
    )
    
    forest_change_var_t3 = GEEVar(
        name=f"forest_tmf_{years[2]}",
        data_type="raster",
        raster_type="categorical",
        gee_images=[forest_tmf_t3],
        aoi=aoi,
        project=project,
        year=years[2],
        tags=["forest"]
    )
    
    forest_change_vars = [forest_change_var_t1, forest_change_var_t2, forest_change_var_t3]

# Convert each GEEVar to local and add as raw
for forest_change_var in forest_change_vars:
    forest_change_var_local = forest_change_var.to_local_raster()
    forest_change_var_local.add_as_raw()

  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/forest_gfc_10_2015.tif, downloaded
✓ Added 'forest_gfc_10_2015' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/forest_gfc_10_2020.tif, downloaded
✓ Added 'forest_gfc_10_2020' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/forest_gfc_10_2024.tif, downloaded
✓ Added 'forest_gfc_10_2024' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


## Rivers

In [33]:
rivers_image = (
    ee.ImageCollection("projects/sat-io/open-datasets/OSM_waterLayer")
    .filterBounds(aoi)
    .mosaic()
    .clip(aoi)
)
rivers_image = rivers_image.gte(2).unmask().clip(aoi).toByte()
rivers_var = GEEVar(
    name="rivers",
    data_type="raster",
    raster_type="categorical",
    gee_images=[rivers_image],
    aoi=aoi,
    project=project
)

rivers_var_local = rivers_var.to_local_raster()
rivers_var_local.add_as_raw();

  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/rivers.tif, downloaded
✓ Added 'rivers' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


## Roads

In [34]:
roads_image = (
    ee.Image(
        "projects/ee-andyarnellgee/assets/crosscutting/infrastructure/roads_osm/roadsAllImageOSM"
    )
    .unmask()
    .clip(aoi)
    .toByte()
)
roads_var = GEEVar(
    name="roads",
    data_type="raster",
    raster_type="categorical",
    gee_images=[roads_image],
    aoi=aoi,
    project=project

)

roads_var_local = roads_var.to_local_raster()
roads_var_local.add_as_raw();

...astructure/roads_osm/roadsAllImageOSM:   0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/roads.tif, downloaded
✓ Added 'roads' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/.venv/lib/python3.11/site-packages/geedim/image.py:254: RuntimeWarning: Couldn't find STAC entry for: 'projects/ee-andyarnellgee/assets/crosscutting/infrastructure/roads_osm/roadsAllImageOSM'.
  return STACClient().get(self.id)


## Towns

In [35]:
jrc_ghsl_pop = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

def closest_epoch(year):
    # Define the list of epochs from 1975 to 2020 with 5-year intervals
    epochs = list(range(1975, 2021, 5))

    # Find the closest epoch to the given year
    closest = min(epochs, key=lambda x: abs(x - year))

    return closest


# Apply the function to each year
closest_epochs = [closest_epoch(y) for y in years]


cities_pop_1 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[0]))
cities_pop_2 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[1]))
cities_pop_3 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[2]))

cities_build_1 = ee.Image(
    "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[0])
).select("built_surface")
cities_build_2 = ee.Image(
    "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[1])
).select("built_surface")
cities_build_3 = ee.Image(
    "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[2])
).select("built_surface")

cities1 = (
    ee.Image(0).where(cities_pop_1.gte(15).And(cities_build_1.gte(90)), 1).clip(aoi)
)
cities2 = (
    ee.Image(0).where(cities_pop_2.gte(15).And(cities_build_2.gte(90)), 1).clip(aoi)
)
cities3 = (
    ee.Image(0).where(cities_pop_3.gte(15).And(cities_build_3.gte(90)), 1).clip(aoi)
)

# Create separate GEEVar for each time period - explicit and clear
towns_var_t1 = GEEVar(
    name=f"towns_{years[0]}",
    data_type="raster",
    raster_type="categorical",
    gee_images=[cities1],
    aoi=aoi,
    project=project,
    year=years[0]
)

towns_var_t2 = GEEVar(
    name=f"towns_{years[1]}",
    data_type="raster",
    raster_type="categorical",
    gee_images=[cities2],
    aoi=aoi,
    project=project,
    year=years[1]
)

towns_var_t3 = GEEVar(
    name=f"towns_{years[2]}",
    data_type="raster",
    raster_type="categorical",
    gee_images=[cities3],
    aoi=aoi,
    project=project,
    year=years[2]
)

towns_vars = [towns_var_t1, towns_var_t2, towns_var_t3]

# Convert each GEEVar to local and add as raw
for towns_var in towns_vars:
    towns_var_local = towns_var.to_local_raster()
    towns_var_local.add_as_raw()

  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/towns_2015.tif, downloaded
✓ Added 'towns_2015' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/towns_2020.tif, downloaded
✓ Added 'towns_2020' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


  0%|          |0/32 tiles [00:00<?]

File /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/towns_2024.tif, downloaded
✓ Added 'towns_2024' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


In [36]:
## Consider using Global Human Modification v3
# https://gee-community-catalog.org/projects/ghm/?h=human

## Oxford accessibility to cities 2015
## ee.Image('Oxford/MAP/accessibility_to_cities_2015_v1_0')


## Custom variables

### GEEVars

In [37]:
new_var = GEEVar(
    name="varname",
    data_type="raster",
    gee_images=[],
    aoi=aoi,
    project=project
)
new_var

GEEVar(name='varname', data_type='raster', year=None, active=True, tags=[], path=None, default_scale=None, default_crs=None, raster_type=None, rasterization_method=None, post_processing=[])

### LocalVector

In [38]:
local_centros_poblados_raster = LocalVectorVar(
    name="centros_poblados",
    path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/capas_crudo/asentamientos/Centrospoblados.shp",
    project=project
).rasterize(
    base=subj_var_local,
    rasterization_method=RasterizationMethod.binary,
).add_as_raw()

✓ Added 'centros_poblados' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


### LocalRaster

In [39]:
from component.script.variables.models import RasterType


coca_var = LocalRasterVar(
    name="coca",
    path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/Variables_raster/DE_COCA.img",
    project=project,
    raster_type=RasterType.categorical
).reproject_and_wrap().add_as_processed()

AttributeError: 'LocalRasterVar' object has no attribute 'reproject_and_wrap'

In [ ]:
#TODO: make an example

## Test Project Save/Load

Test the new Project save and load functionality

In [ ]:
# Save the current project
save_path = project.save()
print(f"Project saved to: {save_path}")
print(f"Variables in project: {list(project.variables.keys())}")

In [ ]:
# Load the project from JSON
loaded_project = Project.load(project_name="nuevo2")
print(f"Loaded project: {loaded_project.project_name}")
print(f"Years: {loaded_project.years}")
print(f"Variables loaded: {list(loaded_project.variables.keys())}")
print(f"\nVariable details:")
for var_name, var in loaded_project.variables.items():
    print(f"  - {var_name}: {type(var).__name__} ({var.data_type})")

In [ ]:
# Test adding more variables to a loaded project
from component.script.variables import LocalRasterVar

# Add a new variable to the loaded project
new_var = LocalRasterVar(
    name="test_variable",
    path=loaded_project.folders.processed_data_folder / "test.tif",
    raster_type="continuous",
    project=loaded_project
)

print(f"Variables after adding new one: {list(loaded_project.variables.keys())}")

# Save again
loaded_project.save()
print("Project saved with new variable!")

## Test Variable Filtering

Try out the new filtering methods

In [ ]:
# 1. Print all variables with their status
print("ALL VARIABLES:")
project.print_variables()

print("\n" + "="*70 + "\n")

# 2. List only active variables
print("ACTIVE VARIABLES:")
project.print_variables(active=True)

print("\n" + "="*70 + "\n")

# 3. List only raster variables
print("RASTER VARIABLES:")
project.print_variables(data_type='raster')

print("\n" + "="*70 + "\n")

# 4. List categorical rasters
print("CATEGORICAL RASTERS:")
project.print_variables(raster_type='categorical')

In [ ]:
project.list_variables()["protected_area"].reproject("EPSG:4326", 30)

In [ ]:
Project.load(project_name="nuevo2")

In [ ]:
project.list_variables(data_type="raster")

In [ ]:
# 5. Count variables
print("VARIABLE COUNTS:")
print(f"Total variables: {project.count_variables()}")
print(f"Active variables: {project.count_variables(active=True)}")
print(f"Inactive variables: {project.count_variables(active=False)}")
print(f"Raster variables: {project.count_variables(data_type='raster')}")
print(f"Vector variables: {project.count_variables(data_type='vector')}")
print(f"Categorical rasters: {project.count_variables(raster_type='categorical')}")
print(f"Continuous rasters: {project.count_variables(raster_type='continuous')}")

In [ ]:
# 6. Get dictionary of specific variables for processing
categorical_rasters = project.list_variables(raster_type='categorical')

print(f"\nFound {len(categorical_rasters)} categorical rasters:")
for name, var in categorical_rasters.items():
    print(f"  • {name}: {var.path}")